In [9]:
# Analysis of haplotype age - infer direction of movement 

In [10]:
%run setup.ipynb
%matplotlib inline
import hapclust

##### NOTES
We need to look at Ghana (e.g. in phase 2) and Mali to see if we see the 'correct' age pattern shows in gam/col samples from the same site/collection date i.e. that tmrca is shorter in col than gam – UNLIKE WHAT WE SEE IN BURKINA - see below

##### Between haplogroups


## setup

In [11]:
# obtain data from unphased callset - only needed for variant annotations
callset = phase1_ar31.callset
pos_all = allel.SortedIndex(callset['2L/variants/POS'])
ann_all = callset['2L/variants/ANN'][:][['Annotation', 'HGVS_p']]

# setup haplotype data
callset_phased = phase1_ar31.callset_phased
genotypes_phased = allel.GenotypeDaskArray(callset_phased['2L/calldata/genotype'])
pos_phased = allel.SortedIndex(callset_phased['2L/variants/POS'])
genotypes_phased.shape, pos_phased.shape

((8296600, 773, 2), (8296600,))

In [12]:
pos_kdr_s = 2422651
pos_kdr_f = 2422652

In [13]:
# define region we're going to analyse
loc_region = pos_phased.locate_range(0, 4000000)
pos_phased_region = pos_phased[loc_region]
pos_phased_region

0,1,2,3,4,...,163958,163959,163960,163961,163962
44688,44691,44732,44736,44756,...,3997372,3997373,3997378,3997381,3997386


In [14]:
# chop genotypes to region, remove colony parents (8 samples) and turn into haplotype array
gen_phased_region = genotypes_phased[loc_region][:, :-8].compute()
gen_phased_region.shape

(163963, 765, 2)

#### grab mvncall

In [15]:
#genotypes
mvn_200 = np.load('../data/phasing_extra_phase1.mvncall.200.npz')
call_mvn = mvn_200['calldata']
gen_mvn = allel.GenotypeArray(call_mvn['genotype'])

#positions
pos_mvn = allel.SortedIndex(mvn_200['variants']['POS'])
pos_mvn

0,1,2
2391228,2400071,2429745


#### weave mvncall

In [16]:
#concatenate old and new gen/pos arrays
haps_combined = np.concatenate([gen_phased_region, gen_mvn], axis=0)
pos_combined = np.concatenate([pos_phased_region, pos_mvn], axis=0)

#sort pos indices
idx_sorted =np.argsort(pos_combined)

#use sorted indices to re-order the combined arrays
haps_combined = allel.GenotypeArray(haps_combined[idx_sorted])

pos_combined = allel.SortedIndex(pos_combined[idx_sorted])

#then turn into haplotype array...
haps = haps_combined.to_haplotypes()
haps

<HaplotypeArray shape=(163966, 1530) dtype=int8>
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0
...
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0

#### tie in annotations

In [17]:
# locate the intersection with unphased callset - needed to tie in annotations
loc1, _ = pos_all.locate_intersection(pos_combined)
np.count_nonzero(loc1)

ann_combined = ann_all[loc1]
ann_combined

collections.Counter(ann_combined['Annotation'])

Counter({b'3_prime_UTR_variant': 2941,
         b'5_prime_UTR_premature_start_codon_': 306,
         b'5_prime_UTR_variant': 1677,
         b'downstream_gene_variant': 18539,
         b'initiator_codon_variant': 3,
         b'intergenic_region': 54849,
         b'intragenic_variant': 48,
         b'intron_variant': 32362,
         b'missense_variant': 5808,
         b'missense_variant&splice_region_var': 70,
         b'splice_acceptor_variant&intron_var': 24,
         b'splice_donor_variant&intron_varian': 27,
         b'splice_region_variant': 36,
         b'splice_region_variant&intron_varia': 649,
         b'splice_region_variant&stop_retaine': 5,
         b'splice_region_variant&synonymous_v': 87,
         b'start_lost': 9,
         b'stop_gained': 37,
         b'stop_lost&splice_region_variant': 4,
         b'stop_retained_variant': 5,
         b'synonymous_variant': 8636,
         b'upstream_gene_variant': 37844})

In [18]:
# perform allele count - needed to locate singletons - NEED TO UP MAX ALLELE TO 2 - MULTI ALLELICS?
ac_combined = haps.count_alleles(max_allele=2)
ac_combined[:1]

,0,1,2
0,1529,1,0


#### setup missense variation

In [19]:
# convenience, define the Vgsc gene region
region_vgsc = SeqFeature('2L', 2358158, 2431617, label='Vgsc')
loc_vgsc = pos_combined.locate_range(region_vgsc.start, region_vgsc.end)
loc_vgsc

slice(24471, 26184, None)

In [20]:
haps_vgsc = haps[loc_vgsc]
ac_vgsc = haps_vgsc.count_alleles(max_allele=2)
ann_vgsc = ann_combined[loc_vgsc]
loc_vgsc_missense = (ann_vgsc['Annotation'] == b'missense_variant') & (ac_vgsc[:, 1] > 7)
np.count_nonzero(loc_vgsc_missense)

19

In [21]:
haps_vgsc_missense = haps_vgsc[loc_vgsc_missense]
lbl_vgsc_missense = [l[2:] for l in ann_vgsc[loc_vgsc_missense]['HGVS_p'].astype('U')]
lbl_vgsc_missense

['Arg254Lys',
 'Val402Leu',
 'Asp466His',
 'Met490Ile',
 'Thr791Met',
 'Leu995Ser',
 'Leu995Phe',
 'Ala1125Val',
 'Ile1527Thr',
 'Asn1570Tyr',
 'Glu1597Gly',
 'Ala1746Ser',
 'Val1853Ile',
 'Ile1868Thr',
 'Pro1874Ser',
 'Pro1874Leu',
 'Phe1920Ser',
 'Ala1934Val',
 'Ile1940Thr']

#### split up haplotype data

In [22]:
# define types of variants to include in EHH analysis - should be mostly neutral
loc_type_neutral = ((ann_combined['Annotation'] == b'intergenic_region') | 
                    (ann_combined['Annotation'] == b'intron_variant') |
                    (ann_combined['Annotation'] == b'downstream_gene_variant') |
                    (ann_combined['Annotation'] == b'upstream_gene_variant') |
                    (ann_combined['Annotation'] == b'synonymous_variant') |
                    (ann_combined['Annotation'] == b'3_prime_UTR_variant') |
                    (ann_combined['Annotation'] == b'5_prime_UTR_variant') 
                    )
np.count_nonzero(loc_type_neutral), loc_type_neutral.shape

(156848, (163966,))

...sort out singletons

In [23]:
# locate singletons - will exclude from EHH analysis
#NB the EHH analysis doesn't need the multiallelics - we've excluded them from the ac_combined so we can use AMs code
loc_sgl_bi = (ac_combined[:,:2].min(axis=1) == 1) & (ac_combined.is_biallelic_01())

loc_nosgl_bi = (ac_combined[:,:2].min(axis=1) > 1) & (ac_combined.is_biallelic_01())
np.count_nonzero(loc_sgl_bi), np.count_nonzero(loc_nosgl_bi), loc_nosgl_bi.shape

(52221, 111612, (163966,))

In [24]:
# these are the variants to use for EHH - missing var are multiallelics
loc_ehh = loc_type_neutral & loc_nosgl_bi
np.count_nonzero(loc_ehh), loc_ehh.shape

(107531, (163966,))

In [25]:
# these are the variants to use for mutational distance
# include non-neutral mutations
loc_mut = loc_sgl_bi | ~loc_type_neutral
np.count_nonzero(loc_mut), loc_mut.shape

(56314, (163966,))

In [26]:
haps_mut = haps[loc_mut]
pos_mut = pos_combined[loc_mut]
haps_mut

<HaplotypeArray shape=(56314, 1530) dtype=int8>
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0
...
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0

In [27]:
haps_ehh = haps[loc_ehh]
pos_ehh = pos_combined[loc_ehh]
#check
pos_mut.locate_key(pos_kdr_s), pos_mut.locate_key(pos_kdr_f)

(10212, 10213)

In [28]:
# read in haplotype metadata to get population
df_haplotypes = phase1_ar31.df_haplotypes
df_haplotypes = df_haplotypes[df_haplotypes.population != 'colony']
df_haplotypes.head()

,label,ox_code,population,label_aug,country,region,sex,m_s,kt_2la,kt_2rb
index,,,,,,,,,,
0,AB0085-Ca,AB0085-C,BFS,"AB0085-Ca [Burkina Faso, Pala, S, F]",Burkina Faso,Pala,F,S,2.0,2.0
1,AB0085-Cb,AB0085-C,BFS,"AB0085-Cb [Burkina Faso, Pala, S, F]",Burkina Faso,Pala,F,S,2.0,2.0
2,AB0087-Ca,AB0087-C,BFM,"AB0087-Ca [Burkina Faso, Bana, M, F]",Burkina Faso,Bana,F,M,2.0,1.0
3,AB0087-Cb,AB0087-C,BFM,"AB0087-Cb [Burkina Faso, Bana, M, F]",Burkina Faso,Bana,F,M,2.0,1.0
4,AB0088-Ca,AB0088-C,BFM,"AB0088-Ca [Burkina Faso, Bana, M, F]",Burkina Faso,Bana,F,M,2.0,0.0


#### split flanks

In [29]:
core_pos = pos_kdr_f
# split the EHH dataset
dist_ehh_right, dist_ehh_left, haps_ehh_right, haps_ehh_left = hapclust.split_flanks(haps_ehh, pos_ehh, core_pos)
# these give the distance from the core position to each downstream variant, moving away from the core
dist_ehh_right

array([     45,     255,     258, ..., 1574721, 1574726, 1574734], dtype=int32)

In [30]:
# split the mutations dataset
dist_mut_right, dist_mut_left, haps_mut_right, haps_mut_left = hapclust.split_flanks(haps_mut, pos_mut, core_pos)
dist_mut_right

array([      0,      82,     353, ..., 1574719, 1574720, 1574729], dtype=int32)

In [31]:
is_accessible = phase1_ar3.accessibility['2L/is_accessible'][:]

#### analyse maximal haplotype sharing 

In [32]:
idx_sorted_right, nspl_right, nspd_right, muts_right = hapclust.neighbour_haplotype_sharing(haps_ehh_right, haps_mut_right, dist_ehh_right, dist_mut_right)
idx_sorted_left, nspl_left, nspd_left, muts_left = hapclust.neighbour_haplotype_sharing(haps_ehh_left, haps_mut_left, dist_ehh_left, dist_mut_left)

# compute accessible lengths - needed for mutation analysis
nspd_right_accessible = hapclust.haplotype_accessible_length(nspd_right, core_pos=core_pos, is_accessible=is_accessible, flank='right')
nspd_left_accessible = hapclust.haplotype_accessible_length(nspd_left, core_pos=core_pos, is_accessible=is_accessible, flank='left')

AttributeError: module 'allel.opt.stats' has no attribute 'neighbour_shared_prefix_lengths_int8'

In [ ]:
# assume constant recombination rate
rr_right = 1.5e-8
# adjust recombination rate on left flank (factor derived from pairwise analysis below)
rr_left = rr_right * 0.37

# assumed mutation rate
mu_right = 3.5e-9
# adjust mutation rate on left flank (factor derived from pairwise analysis below)
mu_left = mu_right * 0.61

In [ ]:
pops_right = df_haplotypes.population[idx_sorted_right]
pop_colors_right = [phase1_ar3.pop_colors[p] for p in pops_right]
pops_left = df_haplotypes.population[idx_sorted_left]
pop_colors_left = [phase1_ar3.pop_colors[p] for p in pops_left]

#### analyse pairwise haplotype sharing

In [ ]:
pspl_right, pspd_right, pmuts_right = hapclust.pairwise_haplotype_sharing(haps_ehh_right, haps_mut_right, dist_ehh_right, dist_mut_right, jitter=False)
pspl_left, pspd_left, pmuts_left = hapclust.pairwise_haplotype_sharing(haps_ehh_left, haps_mut_left, dist_ehh_left, dist_mut_left, jitter=False)
pspd_right_accessible = hapclust.haplotype_accessible_length(pspd_right, core_pos=core_pos, is_accessible=is_accessible, flank='right')
pspd_left_accessible = hapclust.haplotype_accessible_length(pspd_left, core_pos=core_pos, is_accessible=is_accessible, flank='left')

In [ ]:
palette = sns.color_palette(n_colors=11)
sns.palplot(palette);

In [ ]:
#t_hat adjustments
pt_hat_left = (1 + pmuts_left) / (2 * (pspd_left * rr_left + pspd_left_accessible * mu_left))
pt_hat_right = (1 + pmuts_right) / (2 * (pspd_right * rr_right + pspd_right_accessible * mu_right))

In [ ]:
pspd_both = pspd_left + pspd_right
pmuts_both = pmuts_left + pmuts_right
psgl_both = pspd_left * rr_left + pspd_right * rr_right
psml_both = pspd_left_accessible * mu_left + pspd_right_accessible * mu_right
pt_hat_both = (1 + pmuts_both) / (2 * (psgl_both + psml_both))

#### dendrogram clustering

In [ ]:
def plot_dendrogram(dist, cut_height=1e3, yscale='log', ylim=(10, 1e6), linkage_method='average',
                    n_clusters=14):
    """This function plots a dendrogram using scipy and provides some utilities for annotating clusters."""
    
    z = scipy.cluster.hierarchy.linkage(dist, method=linkage_method)

    fig = plt.figure(figsize=(16, 8), )
    gs = mpl.gridspec.GridSpec(nrows=3, ncols=1, height_ratios=[6, .5, 4], hspace=0)

    ax = fig.add_subplot(gs[0])
    sns.despine(ax=ax, offset=3, bottom=True, top=False)
    r = scipy.cluster.hierarchy.dendrogram(
        z, no_labels=True, count_sort=True, 
        color_threshold=0, 
        above_threshold_color='k',
        ax=ax)
    ax.set_ylim(*ylim)
    ax.set_yscale(yscale)
    # ax.set_ylim(bottom=-1000)
    xmin, xmax = ax.xaxis.get_data_interval()
    xticklabels = np.array(list(range(0, len(df_haplotypes), 200)) + [len(df_haplotypes)])
    xticks = xticklabels / len(df_haplotypes)
    xticks = (xticks * (xmax - xmin)) + xmin
    ax.set_xticks(xticks)
    ax.set_xticklabels(xticklabels)
    ax.set_xlabel('Haplotypes')
    ax.xaxis.set_label_position('top')
    ax.set_ylabel('$\hat{t}$', rotation=0, ha='right')

    cluster_palette = sns.color_palette('Set3', n_colors=12)
    if cut_height:
        ax.axhline(cut_height, linestyle='--', color='k')
        # find clusters
        f = scipy.cluster.hierarchy.fcluster(z, cut_height, criterion='distance')
        # compute cluster sizes
        fsz = np.bincount(f)
        # sort largest first
        fsort = np.argsort(fsz)[::-1]
        # take largest n
        fsort = fsort[:n_clusters]
        # get haplotype indices for each cluster
        clusters = [set(np.nonzero(f == i)[0]) for i in fsort]
        clusters_leaves = [sorted([r['leaves'].index(i) for i in cluster])
                           for cluster in clusters]
        ixs = np.argsort([min(cl) for cl in clusters_leaves])
        clusters = [clusters[i] for i in ixs]
        clusters_leaves = [clusters_leaves[i] for i in ixs]
        for i, cluster_leaves in enumerate(clusters_leaves):
            color = cluster_palette[i % len(cluster_palette)]
            x1, x2 = min(cluster_leaves), max(cluster_leaves)
#             ax.axvline(x1*10, color='k', linestyle='--')
#             ax.axvline(x2*10, color='k', linestyle='--')
            ax.fill_between([x1*10, x2*10], 0, cut_height, color=color, alpha=.4, zorder=20)
#             ax.axvspan(x1*10, x2*10, color=color, zorder=-20, alpha=.5)
            ax.text((x1*10 + x2*10) / 2, ylim[0], str(i), ha='center', va='top')
        
    ax = fig.add_subplot(gs[1], )
    sns.despine(ax=ax, left=True, bottom=True)
    pops = df_haplotypes.population[r['leaves']]
    pop_colors = [phase1_ar3.pop_colors[p] for p in pops]
    ax.broken_barh([(i, 1) for i in range(len(df_haplotypes))], yrange=(0, 1), color=pop_colors)
    ax.set_xlim(0, len(df_haplotypes))
    ax.set_yticks([])
    ax.set_ylabel('Population', rotation=0, ha='right', va='center')
    ax.set_xticks([])
    if cut_height:
        for i, cluster_leaves in enumerate(clusters_leaves):
            color = cluster_palette[i % len(cluster_palette)]
            x1, x2 = min(cluster_leaves), max(cluster_leaves)
            ax.axvline(x1, color='k', linestyle='--', zorder=20)
            ax.axvline(x2, color='k', linestyle='--', zorder=20)

    ax = fig.add_subplot(gs[2])
    hapclust.plot_haplotypes(ax, haps_vgsc_missense[:, r['leaves']], lbl_vgsc_missense)
    ax.set_xlim(0, len(df_haplotypes))
    ax.set_xlabel('Haplotypes')
    if cut_height:
        for i, cluster_leaves in enumerate(clusters_leaves):
            color = cluster_palette[i % len(cluster_palette)]
            x1, x2 = min(cluster_leaves), max(cluster_leaves)
            ax.axvline(x1, color='k', linestyle='--', zorder=20)
            ax.axvline(x2, color='k', linestyle='--', zorder=20)
            ax.axvspan(x1, x2, color=color, zorder=20, alpha=.4)

    gs.tight_layout(fig, h_pad=0)
    return clusters

In [ ]:
# canonical dendrogram we'll use for cluster analyses
clusters = plot_dendrogram(pt_hat_both, cut_height=2e3)

# age comparison

In [ ]:
haps_vgsc_missense

In [ ]:
[len(c) for c in clusters]

In [ ]:
# pick these out of the clusters we made earlier
f1 = clusters[2]
f2 = clusters[3]
f3 = clusters[4]
l1 = clusters[5]
l2 = clusters[6]
s1 = clusters[7]
f4 = clusters[8]
f5 = clusters[9]
s2 = clusters[10]
s3 = clusters[12]
s4 = clusters[13]

In [ ]:
# rhg = "resistance haplogroups"
rhg = [f1, f2, f3, f4, f5, s1, s2, s3, s4, l1, l2]
rhg_labels = 'F1 F2 F3 F4 F5 S1 S2 S3 S4 L1 L2'.split()

In [ ]:
for g, l in zip(rhg, rhg_labels):
    print(l)
    print(collections.Counter(df_haplotypes.population[sorted(g)]).most_common())

## F or S

In [ ]:
# compare F haplogroups
def compare_F(ax=None, ylab=True, xlab=True, yticks=True):
    if ax==None:
        fig, ax = plt.subplots()
    else:
        ax = ax
    for g, l, cl in zip(rhg[:5], rhg_labels[:5], palette):
        ixs = allel.condensed_coords_within(g, len(df_haplotypes))
        x = np.log10(pt_hat_both[ixs])
        md = np.median(x)
        #print(l, md)
        sns.distplot(x, bins=np.linspace(1, 3, 20), label=l, hist=False, 
                     kde_kws=dict(linewidth=2, color=cl))
        ax.axvline(md, color=cl, linestyle='--', lw=2)
    if xlab == True:
        ax.set_ylabel('Density')
    ax.set_xlim(1, 4)
    ticks = [1, 2, 3, 4]
    ax.set_xticks(ticks)
    ax.set_title('995F haplogroups')
    ax.set_xticklabels(['$10^{%s}$' % t for t in ticks])
    if ylab == True:
        ax.set_xlabel('$\hat{t}$ (generations)')
        
    ax.set_ylim(0,2)
    if yticks == False:
        ax.yaxis.set_ticklabels([]);
    
compare_F()

In [ ]:
# compare S haplogroups
def compare_S(ax=None, ylab=True, xlab=True, yticks=True):
    if ax == None:
        fig, ax = plt.subplots()
    else:
        ax = ax
    for g, l, cl in zip(rhg[5:9], rhg_labels[5:9], palette):
        ixs = allel.condensed_coords_within(g, len(df_haplotypes))
        x = np.log10(pt_hat_both[ixs])
        md = np.median(x)
        #print(l, md)
        sns.distplot(x, bins=np.linspace(1, 3, 20), label=l, hist=False, 
                     kde_kws=dict(linewidth=2, color=cl))
        ax.axvline(np.median(x), color=cl, linestyle='--', lw=2)
    if xlab == True:
        ax.set_ylabel('Density')
    
    ax.set_xlim(1, 4)
    ticks = [1, 2, 3, 4]
    ax.set_xticks(ticks)
    ax.set_title('995S haplogroups')
    ax.set_xticklabels(['$10^{%s}$' % t for t in ticks])

    if ylab == True:
        ax.set_xlabel('$\hat{t}$ (generations)')
    ax.set_ylim(0,2)
    if yticks == False:
        ax.yaxis.set_ticklabels([]);
compare_S()

In [ ]:
# compare S haplogroups
def compare_L(ax=None, ylab=True, xlab=True, yticks=True):
    if ax == None:
        fig, ax = plt.subplots()
    else:
        ax = ax
    for g, l, cl in zip(rhg[9:], rhg_labels[9:], palette):
        ixs = allel.condensed_coords_within(g, len(df_haplotypes))
        x = np.log10(pt_hat_both[ixs])
        md = np.median(x)
        #print(l, md)
        sns.distplot(x, bins=np.linspace(1, 3, 20), label=l, hist=False, 
                     kde_kws=dict(linewidth=2, color=cl))
        ax.axvline(np.median(x), color=cl, linestyle='--', lw=2)
    if xlab == True:
        ax.set_ylabel('Density')
    ax.set_xlim(1, 4)
    ticks = [1, 2, 3, 4]
    ax.set_xticks(ticks)
    ax.set_title('995L haplogroups')
    ax.set_xticklabels(['$10^{%s}$' % t for t in ticks])
    ax.set_ylim(0,2)
    if ylab == True:
        ax.set_xlabel('$\hat{t}$ (generations)')
    if yticks == False:
        ax.yaxis.set_ticklabels([]);
compare_L()

In [ ]:
#all?
fig, ax = plt.subplots()
for g, l, cl in zip(rhg, rhg_labels, palette):
    ixs = allel.condensed_coords_within(g, len(df_haplotypes))
    x = np.log10(pt_hat_both[ixs])
    md = np.median(x)
    print(l, md)
    sns.distplot(x, bins=np.linspace(1, 3, 20), label=l, hist=False, 
                 kde_kws=dict(linewidth=2, color=cl))
    ax.axvline(np.median(x), color=cl, linestyle='--', lw=2)
ax.set_ylabel('Density')
ax.set_xlim(1, 4)
ticks = [1, 2, 3, 4]
ax.set_xticks(ticks)
ax.set_xticklabels(['$10^{%s}$' % t for t in ticks])
ax.set_xlabel('$\hat{t}$ (generations)');

## within haplogroups

In [ ]:
#bootstrap the mean t_hat
def bootstrap(x, n, f):
    dtype = np.array(f(x)).dtype
    out = np.zeros(n, dtype=dtype)
    for i in range(n):
        ix = np.random.choice(x.shape[0], size=x.shape[0], replace=True)
        out[i] = f(x[ix])
    return out

In [ ]:
def distplot_cluster_pop_t_hat(cluster, title, ax=None, xlab=True, yticks=True):
    cluster = list(cluster)
    pops_clst = df_haplotypes.population[cluster]
    pops_unique = pops_clst.unique()
    pops_subclusters = [np.array(cluster)[(pops_clst == p).values] for p in pops_unique]
    
    if ax == None:
        fig, ax = plt.subplots()
    else:
        ax = ax
    for g, l in zip(pops_subclusters, pops_unique):
        ixs = allel.condensed_coords_within(g, len(df_haplotypes))
        x = np.log10(pt_hat_both[ixs])
        color = phase1_ar3.pop_colors[l]
        md = np.median(x)
        bs = bootstrap(x, 1000, np.median)
        lbl = '%s (%.0f; 95%% CI [%.0f, %.0f])' % (l, 10**md, 
                                               10**np.percentile(bs, 2.5), 
                                               10**np.percentile(bs, 97.5))
        sns.distplot(x, bins=np.linspace(1, 3, 20), label=lbl, hist=False, 
                     kde_kws=dict(linewidth=2, color=color, label=lbl))
        ax.axvspan(np.percentile(bs, 2.5), np.percentile(bs, 97.5), 
                   color=color, alpha=.3)
        ax.axvline(md, color=color, lw=2, linestyle='--')
    
    ax.set_xlabel('$\hat{t}$')
    if xlab == True:
        ax.set_ylabel('Density')
    ax.set_xlim(1, 4)
    ticks = [1, 2, 3, 4]
    ax.set_xticks(ticks)
    ax.set_xticklabels(['$10^{%s}$' % t for t in ticks])
    ax.set_xlabel('$\hat{t}$ (generations)')
    ax.set_title(title)
    ax.legend(bbox_to_anchor=(1, 1), loc='upper right')
    ax.set_ylim(0,2)
    if yticks == False:
        ax.yaxis.set_ticklabels([])
        ##ax.yaxis.set_major_locator(plt.NullLocator());
    #fig.tight_layout()

In [ ]:
distplot_cluster_pop_t_hat(f3, 'Haplogroup F3')

In [ ]:
distplot_cluster_pop_t_hat(f5, 'Haplogroup F5')

In [ ]:
distplot_cluster_pop_t_hat(s4, 'Haplogroup S4')

### create composite figure

In [ ]:
fig = plt.figure(figsize=(16, 6), )
gs = mpl.gridspec.GridSpec(nrows=2, ncols=3)

ax = fig.add_subplot(gs[0,0])
compare_F(ax=ax, ylab=False)
ax = fig.add_subplot(gs[0,1])
compare_S(ax=ax, ylab=False, xlab=False, yticks=False)
ax = fig.add_subplot(gs[0,2])
compare_L(ax=ax, ylab=False, xlab=False, yticks=False)
ax = fig.add_subplot(gs[1,0])
distplot_cluster_pop_t_hat(f3, 'Haplogroup F3', ax=ax)
ax = fig.add_subplot(gs[1,1])
distplot_cluster_pop_t_hat(f5, 'Haplogroup F5', ax=ax, xlab=False, yticks=False)
ax = fig.add_subplot(gs[1,2])
distplot_cluster_pop_t_hat(s4, 'Haplogroup S4', ax=ax, xlab=False, yticks=False)

gs.tight_layout(fig)
fn = '../artwork/Figure_3_RelativeAges.jpeg'
fig.savefig(fn, jpeg_quality=100, dpi=100, bbox_inches='tight')

In [ ]:
##### NOTES
We need to look at Ghana and Mali to see if we see the 'correct' age pattern shows in gam/col samples from the same site/collection date i.e. that tmrca is shorter in col than gam – UNLIKE WHAT WE SEE IN BURKINA - see below

##### Between haplogroups

# Stuff for Martin

In [ ]:
snp = p3r.locate_range(8564154, 8564160)
snp

In [ ]:
md_call = phase2_ar1.callset

g3r = allel.GenotypeDaskArray(md_call['3R/calldata/genotype'])
p3r = allel.SortedIndex(md_call['3R/variants/POS'])

ref = md_call['3R/variants/REF'][:]
alt = md_call['3R/variants/ALT'][:]
aac = md_call['3R/variants/AC'][:]

ref_ = ref[snp]
alt_ = alt[snp]
aac_ = aac[snp]

aac_

In [ ]:
p_reg = p3r[snp]
p_reg

In [ ]:
g_reg =g3r[snp]
g_reg

In [ ]:
ac = g_reg.count_alleles()
ac

In [ ]:
#per pop?
ac_f = g_reg.count_alleles_subpops()
df_p2 = pandas.read_csv('~/Git/agam-vgsc-report/data/phase2_samples.meta.txt', delimiter='\t')

pop_dict = {}
for p in phase2_ar1.pop_ids:
    pop_dict[p] = df_p2[df_p2['population'] == p].index.tolist()

ac_pop = g_reg.count_alleles_subpops(pop_dict)

for p in ac_pop:
    print(p)
    print(ac_pop[p][1:2])